In [ ]:
!pip install transformers
!pip install eli5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107719 sha256=81a216bf93ad70834f7eabffc7ba52bec8ac54fd0ee7f30fa51114015655ec22
  Stored in directory: /root/.cache/pip/wheels/b8/58/ef/2cf4c306898c2338d51540e0922c8e0d6028e07007085c0004
Successfully built eli5


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/GPT

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/GPT


In [ ]:
# Import generic wrappers
from transformers import AutoModel, AutoTokenizer , AutoModelForSequenceClassification, pipeline
import numpy as np
import pandas as pd

### Provide the input file name
#input_text_file = "DagPap22_Corpus.csv"

datasetname = [ 'DagPap22','chatgpt_human_combined','IELTS_GPT35_Human_Comparison']
robertamodelname = 'roberta'

def getInputfileName(datasetname):
    return f"./output/{datasetname}_Corpus.csv"

def getOutputfileName(datasetname, modelname):
    return f"./output/{datasetname}_result_{modelname}.csv"

# Define the model repo
model_name = "roberta-base-openai-detector"

# Download pytorch model
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def processRobertaModel(datasetFile):
    print(f"Processing for datasets : {datasetFile}")
    df_combined = pd.read_csv(getInputfileName(datasetFile))
    df_delta=pd.DataFrame(columns=['label', 'score'])
    for txt in df_combined['text']:
      pred_pipeline  = pipeline("text-classification", model="roberta-base-openai-detector")
      # print(len(my_pipeline.tokenizer.tokenize(input_text)))
      df_delta=df_delta.append(pred_pipeline(txt, truncation=True))

    df_combined.reset_index(drop=True, inplace=True)
    df_delta.reset_index(drop=True, inplace=True)
    df_final= pd.concat([df_combined, df_delta], axis=1)
    df_final['label'] = df_final['label'].map({'Fake':'chatgpt', 'Real':'human'})

    print(classification_report(df_combined['source'], df_final['label']))
    print(confusion_matrix(df_combined['source'], df_final['label']))

    df_final.to_csv(getOutputfileName(datasetFile,robertamodelname))

In [ ]:
#Process for  all datasets one by one
for datasetfile in datasetname:
    processRobertaModel(datasetfile)

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
<ipython-input-6-9e9b42e7dcfb>:9: FutureWarning: The frame.append metho

              precision    recall  f1-score   support

     chatgpt       0.91      0.98      0.95       116
       human       0.98      0.91      0.94       116

    accuracy                           0.94       232
   macro avg       0.95      0.94      0.94       232
weighted avg       0.95      0.94      0.94       232

[[114   2]
 [ 11 105]]


Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-6-9e9b42e7dcfb>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_delta=df_delta.append(pred_pipeline(txt, truncation=True))
Some weights of the model checkpoint at roberta-base-openai-detector were 

              precision    recall  f1-score   support

     chatgpt       0.95      0.98      0.96       126
       human       0.98      0.94      0.96       126

    accuracy                           0.96       252
   macro avg       0.97      0.96      0.96       252
weighted avg       0.97      0.96      0.96       252

[[124   2]
 [  7 119]]


Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-6-9e9b42e7dcfb>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_delta=df_delta.append(pred_pipeline(txt, truncation=True))
Some weights of the model checkpoint at roberta-base-openai-detector were 

              precision    recall  f1-score   support

     chatgpt       0.00      0.00      0.00         0
       human       1.00      0.97      0.99       183

    accuracy                           0.97       183
   macro avg       0.50      0.49      0.49       183
weighted avg       1.00      0.97      0.99       183

[[  0   0]
 [  5 178]]
